# Thai-Winograd

## Proprocessing

In [ ]:
ls

drive/  sample_data/


In [ ]:
!pip install tqdm

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load a dataset
dataset_en = load_dataset('winograd_wsc','wsc285')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for winograd_wsc contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/winograd_wsc.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating test split:   0%|          | 0/285 [00:00<?, ? examples/s]

In [ ]:
dataset_en

DatasetDict({
    test: Dataset({
        features: ['text', 'pronoun', 'pronoun_loc', 'quote', 'quote_loc', 'options', 'label', 'source'],
        num_rows: 285
    })
})

In [ ]:
# Function to create Winograd schema prompt
def create_winograd_prompt(data):
    # Extract necessary information
    text = data['text']
    quote = data['quote']
    pronoun = data['pronoun']
    options = data['options']

    # Create the prompt with bold pronoun
    prompt = f"""{text.replace(pronoun, f'**{pronoun}**')}\nSnippet: {quote.replace(pronoun, f'**{pronoun}**')}\nOptions:\n{options[0]}\n{options[1]}"""

    return prompt.strip()

# Process the dataset and generate prompts
def process_dataset(dataset):
    prompts = []
    for entry in dataset['test']:
        prompt = create_winograd_prompt(entry)
        prompts.append(prompt)
    return prompts

In [ ]:
prompt_english=process_dataset(dataset_en)

In [ ]:
prompt_english[0]

'The city councilmen refused the demonstrators a permit because **they** feared violence.\nSnippet: **they** feared violence\nOptions:\nThe city councilmen\nThe demonstrators'

In [ ]:
label_english=dataset_en['test']['label']

In [ ]:
option_english=dataset_en['test']['options']

In [ ]:
from datasets import load_dataset

# Load a dataset
dataset_th = load_dataset('pakphum/winograd_th')


Generating test split:   0%|          | 0/285 [00:00<?, ? examples/s]

In [ ]:
label_thai=dataset_th['test']['label']

In [ ]:
prompt_thai=process_dataset(dataset_th)

In [ ]:
option_thai=dataset_th['test']['options']

In [ ]:
prompt_thai[0]

'สมาชิกสภาเทศบาลเมืองปฏิเสธใบอนุญาตผู้ชุมนุมเพราะ**พวกเขา**กลัวความรุนแรง\nSnippet: **พวกเขา**กลัวความรุนแรง\nOptions:\nสมาชิกสภาเทศบาลเมือง\nผู้ชุมนุม'

In [ ]:
from datasets import load_dataset

dataset_th_exact = load_dataset("pakphum/winograd_th_exact_translation")

In [ ]:
label_thai_exact=dataset_th_exact['test']['label']

In [ ]:
prompt_thai_exact=process_dataset(dataset_th_exact)

In [ ]:
option_thai_exact=dataset_th_exact['test']['options']

In [ ]:
prompt_thai_exact[0]

'สมาชิกสภาเมืองปฏิเสธที่จะให้ใบอนุญาตแก่ผู้ประท้วงเนื่องจาก**พวกเขา**เกรงกลัวความรุนแรง\nSnippet: **พวกเขา**เกรงกลัวความรุนแรง\nOptions:\nสมาชิกสภาเทศบาล\nผู้ประท้วง'

In [ ]:
system_prompt_en = "You will be provided with a sentence and a snippet containing a pronoun enclosed in asterisks (**). Your task is to determine the correct referent of the pronoun from the given options. Respond only with one of the provided choices, exactly as it is written. For example, if the options are 'The city councilmen' and 'The demonstrators', respond only with 'The city councilmen' or 'The demonstrators'."
system_prompt_th = "You will be provided with a sentence and a snippet containing a pronoun enclosed in asterisks (**). Your task is to determine the correct referent of the pronoun from the given options. Respond only with one of the provided choices, exactly as it is written. For example, if the options are 'The city councilmen' and 'The demonstrators', respond only with 'The city councilmen' or 'The demonstrators'."
system_prompt_th_exact = "You will be provided with a sentence and a snippet containing a pronoun enclosed in asterisks (**). Your task is to determine the correct referent of the pronoun from the given options. Respond only with one of the provided choices, exactly as it is written. For example, if the options are 'The city councilmen' and 'The demonstrators', respond only with 'The city councilmen' or 'The demonstrators'."

## Typhoon

In [ ]:
!pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
def ask_typhoon(model,systemPrompt,userPrompt):

    try:
        # Make an API call to OpenAI
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": systemPrompt},
                {"role": "user", "content": userPrompt}
            ],
            temperature=0,
            seed=77
          )
        return response.choices[0].message
    except Exception as e:
        return f"An error occurred: {e}"


In [ ]:
import csv
from openai import OpenAI
import os

client = OpenAI(
    api_key='YOUR_KEY',
    base_url="https://api.opentyphoon.ai/v1",
)

# Function to test the model on a specific dataset
def test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name):
    ans_typhoon = []  # List to store generated responses
    results = []  # List to store results (answer, gold label, and whether it was correct)
    correct = 0  # Counter for correctly matched responses
    total = len(prompts)  # Total number of attempts

    for i, p in enumerate(prompts):
        # Call the model to get a response
        ret = ask_typhoon(model_name, system_prompt, p).content
        ans_typhoon.append(ret)  # Store the generated response

        # Determine the gold label based on the correct option
        if labels[i] == 0:
            gold_label = options[i][0]
        else:
            gold_label = options[i][1]

        # Check if the generated response matches the gold label
        is_correct = int(ret == gold_label)
        correct += is_correct  # Increment correct count if there's a match
        results.append((dataset_name, ret, gold_label, is_correct))  # Store result (dataset, answer, gold label, correct/incorrect)

    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0

    # Print and return the results
    print(f"Dataset: {dataset_name}, Accuracy: {accuracy * 100:.2f}%")
    return results, accuracy

# Dataset configurations
datasets = [
    {"name": "dataset_en", "prompts": prompt_english, "options": option_english, "labels": label_english},
    {"name": "dataset_th", "prompts": prompt_thai, "options": option_thai, "labels": label_thai},
    {"name": "dataset_th_exact", "prompts": prompt_thai_exact, "options": option_thai_exact, "labels": label_thai_exact},
]

model_name = "typhoon-instruct"
system_prompt = system_prompt_en

# Initialize a CSV file for each model that will hold the results from all datasets
with open(f'{model_name}_results.csv', mode='w', newline='', encoding='utf-8') as model_results_file:
    csv_writer = csv.writer(model_results_file)

    # Write the header of the CSV file
    csv_writer.writerow(['Dataset', 'Model Answer', 'Gold Label', 'Correct'])

    # Loop through each dataset and test the model
    for dataset in datasets:
        dataset_name = dataset["name"]
        prompts = dataset["prompts"]
        options = dataset["options"]
        labels = dataset["labels"]

        # Test the model on the current dataset
        results, accuracy = test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name)

        # Write the dataset results to the CSV file
        for result in results:
            csv_writer.writerow(result)

print(f"Results have been saved to {model_name}_results.csv")

Dataset: dataset_en, Accuracy: 48.42%
Dataset: dataset_th, Accuracy: 55.09%
Dataset: dataset_th_exact, Accuracy: 53.33%
Results have been saved to typhoon-instruct_results.csv


## GPT family

### GPT-4

In [ ]:
!pip install openai


In [ ]:
def ask_gpt(model,systemPrompt,userPrompt):

    try:
        # Make an API call to OpenAI
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": systemPrompt},
                {"role": "user", "content": userPrompt}
            ],
            temperature=0,
            seed=77
        )
        return response.choices[0].message
    except Exception as e:
        return f"An error occurred: {e}"


In [ ]:
import csv
from openai import OpenAI
import os

client = OpenAI(api_key='YOUR_KEY')

# Function to test the model on a specific dataset
def test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name):
    ans_gpt = []  # List to store generated responses
    results = []  # List to store results (answer, gold label, and whether it was correct)
    correct = 0  # Counter for correctly matched responses
    total = len(prompts)  # Total number of attempts

    for i, p in enumerate(prompts):
        # Call the model to get a response
        ret = ask_gpt(model_name, system_prompt, p).content
        ans_gpt.append(ret)  # Store the generated response

        # Determine the gold label based on the correct option
        if labels[i] == 0:
            gold_label = options[i][0]
        else:
            gold_label = options[i][1]

        # Check if the generated response matches the gold label
        is_correct = int(ret == gold_label)
        correct += is_correct  # Increment correct count if there's a match
        results.append((dataset_name, ret, gold_label, is_correct))  # Store result (dataset, answer, gold label, correct/incorrect)

    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0

    # Print and return the results
    print(f"Dataset: {dataset_name}, Accuracy: {accuracy * 100:.2f}%")
    return results, accuracy

# Dataset configurations
datasets = [
    {"name": "dataset_en", "prompts": prompt_english, "options": option_english, "labels": label_english},
    {"name": "dataset_th", "prompts": prompt_thai, "options": option_thai, "labels": label_thai},
    {"name": "dataset_th_exact", "prompts": prompt_thai_exact, "options": option_thai_exact, "labels": label_thai_exact},
]

model_name = "gpt-4"
system_prompt = system_prompt_en

# Initialize a CSV file for each model that will hold the results from all datasets
with open(f'{model_name}_results.csv', mode='w', newline='', encoding='utf-8') as model_results_file:
    csv_writer = csv.writer(model_results_file)

    # Write the header of the CSV file
    csv_writer.writerow(['Dataset', 'Model Answer', 'Gold Label', 'Correct'])

    # Loop through each dataset and test the model
    for dataset in datasets:
        dataset_name = dataset["name"]
        prompts = dataset["prompts"]
        options = dataset["options"]
        labels = dataset["labels"]

        # Test the model on the current dataset
        results, accuracy = test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name)

        # Write the dataset results to the CSV file
        for result in results:
            csv_writer.writerow(result)

print(f"Results have been saved to {model_name}_results.csv")

Dataset: dataset_en, Accuracy: 94.04%
Dataset: dataset_th, Accuracy: 75.09%
Dataset: dataset_th_exact, Accuracy: 79.65%
Results have been saved to gpt-4_results.csv


### GPT 3.5

In [ ]:
import csv
from openai import OpenAI
import os

client = OpenAI(api_key='YOUR_KEY')

# Function to test the model on a specific dataset
def test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name):
    ans_gpt = []  # List to store generated responses
    results = []  # List to store results (answer, gold label, and whether it was correct)
    correct = 0  # Counter for correctly matched responses
    total = len(prompts)  # Total number of attempts

    for i, p in enumerate(prompts):
        # Call the model to get a response
        ret = ask_gpt(model_name, system_prompt, p).content
        ans_gpt.append(ret)  # Store the generated response

        # Determine the gold label based on the correct option
        if labels[i] == 0:
            gold_label = options[i][0]
        else:
            gold_label = options[i][1]

        # Check if the generated response matches the gold label
        is_correct = int(ret == gold_label)
        correct += is_correct  # Increment correct count if there's a match
        results.append((dataset_name, ret, gold_label, is_correct))  # Store result (dataset, answer, gold label, correct/incorrect)

    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0

    # Print and return the results
    print(f"Dataset: {dataset_name}, Accuracy: {accuracy * 100:.2f}%")
    return results, accuracy

# Dataset configurations
datasets = [
    {"name": "dataset_en", "prompts": prompt_english, "options": option_english, "labels": label_english},
    {"name": "dataset_th", "prompts": prompt_thai, "options": option_thai, "labels": label_thai},
    {"name": "dataset_th_exact", "prompts": prompt_thai_exact, "options": option_thai_exact, "labels": label_thai_exact},
]

model_name = "gpt-3.5-turbo-0125"
system_prompt = system_prompt_en

# Initialize a CSV file for each model that will hold the results from all datasets
with open(f'{model_name}_results.csv', mode='w', newline='', encoding='utf-8') as model_results_file:
    csv_writer = csv.writer(model_results_file)

    # Write the header of the CSV file
    csv_writer.writerow(['Dataset', 'Model Answer', 'Gold Label', 'Correct'])

    # Loop through each dataset and test the model
    for dataset in datasets:
        dataset_name = dataset["name"]
        prompts = dataset["prompts"]
        options = dataset["options"]
        labels = dataset["labels"]

        # Test the model on the current dataset
        results, accuracy = test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name)

        # Write the dataset results to the CSV file
        for result in results:
            csv_writer.writerow(result)

print(f"Results have been saved to {model_name}_results.csv")

Dataset: dataset_en, Accuracy: 70.88%
Dataset: dataset_th, Accuracy: 52.63%
Dataset: dataset_th_exact, Accuracy: 52.28%
Results have been saved to gpt-3.5-turbo-0125_results.csv


## Claude

In [ ]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.9/891.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [ ]:
def ask_claude(model,systemPrompt,userPrompt):

    try:
        # Make an API call to OpenAI
        response = client.messages.create(
            model=model,
            max_tokens=50,
            temperature=0.0,
            system=systemPrompt,
            messages=[
                {"role": "user", "content": userPrompt}
            ],
        )
        return response.content
    except Exception as e:
        return f"An error occurred: {e}"


In [ ]:
import csv
import os
import anthropic
import time
import requests

client = anthropic.Anthropic(
    api_key="API_KEY"
)

# Function to test the model on a specific dataset
def test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name):
    ans_claude = []  # List to store generated responses
    results = []  # List to store results (answer, gold label, and whether it was correct)
    correct = 0  # Counter for correctly matched responses
    total = len(prompts)  # Total number of attempts

    for i, p in enumerate(prompts):
        # Call the model to get a response
        time.sleep(8)
        ret = ask_claude(model_name, system_prompt, p)[0].text
        ans_claude.append(ret)  # Store the generated response

        # Determine the gold label based on the correct option
        if labels[i] == 0:
            gold_label = options[i][0]
        else:
            gold_label = options[i][1]

        # Check if the generated response matches the gold label
        is_correct = int(ret == gold_label)
        correct += is_correct  # Increment correct count if there's a match
        results.append((dataset_name, ret, gold_label, is_correct))  # Store result (dataset, answer, gold label, correct/incorrect)

    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0

    # Print and return the results
    print(f"Dataset: {dataset_name}, Accuracy: {accuracy * 100:.2f}%")
    return results, accuracy

# Dataset configurations
datasets = [
    {"name": "dataset_en", "prompts": prompt_english, "options": option_english, "labels": label_english},
    {"name": "dataset_th", "prompts": prompt_thai, "options": option_thai, "labels": label_thai},
    {"name": "dataset_th_exact", "prompts": prompt_thai_exact, "options": option_thai_exact, "labels": label_thai_exact},
]

model_name = "claude-3-haiku-20240307"
system_prompt = system_prompt_en

# Initialize a CSV file for each model that will hold the results from all datasets
with open(f'{model_name}_results.csv', mode='w', newline='', encoding='utf-8') as model_results_file:
    csv_writer = csv.writer(model_results_file)

    # Write the header of the CSV file
    csv_writer.writerow(['Dataset', 'Model Answer', 'Gold Label', 'Correct'])

    # Loop through each dataset and test the model
    for dataset in datasets:
        dataset_name = dataset["name"]
        prompts = dataset["prompts"]
        options = dataset["options"]
        labels = dataset["labels"]

        # Test the model on the current dataset
        results, accuracy = test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name)

        # Write the dataset results to the CSV file
        for result in results:
            csv_writer.writerow(result)

print(f"Results have been saved to {model_name}_results.csv")

Dataset: dataset_en, Accuracy: 64.21%
Dataset: dataset_th, Accuracy: 51.93%
Dataset: dataset_th_exact, Accuracy: 57.19%
Results have been saved to claude-3-haiku-20240307_results.csv


In [ ]:
import csv
import os
import anthropic
import time
import requests

client = anthropic.Anthropic(
    api_key="API_KEY"
)

# Function to test the model on a specific dataset
def test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name):
    ans_claude = []  # List to store generated responses
    results = []  # List to store results (answer, gold label, and whether it was correct)
    correct = 0  # Counter for correctly matched responses
    total = len(prompts)  # Total number of attempts

    for i, p in enumerate(prompts):
        # Call the model to get a response
        time.sleep(8)
        ret = ask_claude(model_name, system_prompt, p)[0].text
        ans_claude.append(ret)  # Store the generated response

        # Determine the gold label based on the correct option
        if labels[i] == 0:
            gold_label = options[i][0]
        else:
            gold_label = options[i][1]

        # Check if the generated response matches the gold label
        is_correct = int(ret == gold_label)
        correct += is_correct  # Increment correct count if there's a match
        results.append((dataset_name, ret, gold_label, is_correct))  # Store result (dataset, answer, gold label, correct/incorrect)

    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0

    # Print and return the results
    print(f"Dataset: {dataset_name}, Accuracy: {accuracy * 100:.2f}%")
    return results, accuracy

# Dataset configurations
datasets = [
    {"name": "dataset_en", "prompts": prompt_english, "options": option_english, "labels": label_english},
    {"name": "dataset_th", "prompts": prompt_thai, "options": option_thai, "labels": label_thai},
    {"name": "dataset_th_exact", "prompts": prompt_thai_exact, "options": option_thai_exact, "labels": label_thai_exact},
]

model_name = "claude-3-sonnet-20240229"
system_prompt = system_prompt_en

# Initialize a CSV file for each model that will hold the results from all datasets
with open(f'{model_name}_results.csv', mode='w', newline='', encoding='utf-8') as model_results_file:
    csv_writer = csv.writer(model_results_file)

    # Write the header of the CSV file
    csv_writer.writerow(['Dataset', 'Model Answer', 'Gold Label', 'Correct'])

    # Loop through each dataset and test the model
    for dataset in datasets:
        dataset_name = dataset["name"]
        prompts = dataset["prompts"]
        options = dataset["options"]
        labels = dataset["labels"]

        # Test the model on the current dataset
        results, accuracy = test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name)

        # Write the dataset results to the CSV file
        for result in results:
            csv_writer.writerow(result)

print(f"Results have been saved to {model_name}_results.csv")

Dataset: dataset_en, Accuracy: 80.70%
Dataset: dataset_th, Accuracy: 64.91%
Dataset: dataset_th_exact, Accuracy: 65.96%
Results have been saved to claude-3-sonnet-20240229_results.csv


In [ ]:
import csv
import os
import anthropic
import time
import requests

client = anthropic.Anthropic(
    api_key="API_KEY"
)

# Function to test the model on a specific dataset
def test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name):
    ans_claude = []  # List to store generated responses
    results = []  # List to store results (answer, gold label, and whether it was correct)
    correct = 0  # Counter for correctly matched responses
    total = len(prompts)  # Total number of attempts

    for i, p in enumerate(prompts):
        # Call the model to get a response
        time.sleep(8)
        ret = ask_claude(model_name, system_prompt, p)[0].text
        ans_claude.append(ret)  # Store the generated response

        # Determine the gold label based on the correct option
        if labels[i] == 0:
            gold_label = options[i][0]
        else:
            gold_label = options[i][1]

        # Check if the generated response matches the gold label
        is_correct = int(ret == gold_label)
        correct += is_correct  # Increment correct count if there's a match
        results.append((dataset_name, ret, gold_label, is_correct))  # Store result (dataset, answer, gold label, correct/incorrect)

    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0

    # Print and return the results
    print(f"Dataset: {dataset_name}, Accuracy: {accuracy * 100:.2f}%")
    return results, accuracy

# Dataset configurations
datasets = [
    {"name": "dataset_en", "prompts": prompt_english, "options": option_english, "labels": label_english},
    {"name": "dataset_th", "prompts": prompt_thai, "options": option_thai, "labels": label_thai},
    {"name": "dataset_th_exact", "prompts": prompt_thai_exact, "options": option_thai_exact, "labels": label_thai_exact},
]

model_name = "claude-3-opus-20240229"
system_prompt = system_prompt_en

# Initialize a CSV file for each model that will hold the results from all datasets
with open(f'{model_name}_results.csv', mode='w', newline='', encoding='utf-8') as model_results_file:
    csv_writer = csv.writer(model_results_file)

    # Write the header of the CSV file
    csv_writer.writerow(['Dataset', 'Model Answer', 'Gold Label', 'Correct'])

    # Loop through each dataset and test the model
    for dataset in datasets:
        dataset_name = dataset["name"]
        prompts = dataset["prompts"]
        options = dataset["options"]
        labels = dataset["labels"]

        # Test the model on the current dataset
        results, accuracy = test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name)

        # Write the dataset results to the CSV file
        for result in results:
            csv_writer.writerow(result)

print(f"Results have been saved to {model_name}_results.csv")

Dataset: dataset_en, Accuracy: 92.63%
Dataset: dataset_th, Accuracy: 77.89%
Dataset: dataset_th_exact, Accuracy: 84.21%
Results have been saved to claude-3-opus-20240229_results.csv


In [ ]:
import anthropic
import time
import requests

client = anthropic.Anthropic(
    api_key="API_KEY"
)

ans = []  # List to store generated responses
correct = 0  # Counter for correctly matched responses
all = 0  # Counter for all attempts
system_prompt = system_prompt_en
model = "claude-3-opus-20240229"
for i, p in enumerate(prompt_english):
    time.sleep(5)
    ret = ask_claude(model,system_prompt,p)[0].text
    ans.append(ret)  # Store the generated response
    print(ret)
    # Determine the gold label based on the correct option in wsc_th
    if label_english[i] == 0:
        gold_label = option_english[i][0]
    else:
        gold_label = option_english[i][1]

    # Check if the generated response matches the gold label
    if ret == gold_label:
        correct += 1  # Increment correct count if there's a match

    all += 1  # Increment total attempts

# Calculate accuracy as the number of correct answers divided by the total number of attempts
accuracy = correct / all if all > 0 else 0  # Ensure division by zero is handled

print(f"Accuracy: {accuracy*100:.2f}%")


The city councilmen
The demonstrators
the trophy
the suitcase
Joan
Susan
Paul
George
the witness
the witness
the delivery truck
the school bus
Frank
Bill
The man
The son
The large ball
The table
John
Billy
Tom
Tom
Sue
Sally
The sculpture
The shelf
Sam's drawing
Sam's drawing
Anna
Lucy
The firemen
The police
Frank
Tom
Jim
Kevin
The sack of potatoes
The sack of potatoes
Martin
Martin
The older students
The younger students
the bottle
the cup
Susan
Ann
Sid
Mark
Susan
Ann
Joe
Joe's uncle
The painting
The oak tree
The gap
The wall
The drain
The hair
The meeting
The train
The pillar
The stage
The announcement
The subway
The concert
The rain
The rag
The knife
Ann
Mary
The water bottle
The backpack
The pot
The shelf
The map
The building
Bob
Charlie
Bob
Charlie
Bob
Bob
Bob
Bob
The con artist
Sam
The dog
The spot under the tree
The cat
The mouse
Anne
Anne's daughter
Alice
Alice's daughter
Jim
the guy in uniform
The fish
The worm
The key
The chewing gum
The dog
The cat
The tree
The roof
The custo

In [ ]:
import anthropic
import time
import requests

client = anthropic.Anthropic(
    api_key="API_KEY"
)

ans = []  # List to store generated responses
correct = 0  # Counter for correctly matched responses
all = 0  # Counter for all attempts
system_prompt = system_prompt_th
model = "claude-3-haiku-20240307"
for i, p in enumerate(prompt_thai):
    time.sleep(5)
    ret = ask_claude(model,system_prompt,p)[0].text
    print(ret)
    ans.append(ret)  # Store the generated response

    # Determine the gold label based on the correct option in wsc_th
    if label_thai[i] == 0:
        gold_label = option_thai[i][0]
    else:
        gold_label = option_thai[i][1]

    # Check if the generated response matches the gold label
    if ret == gold_label:
        correct += 1  # Increment correct count if there's a match

    all += 1  # Increment total attempts

# Calculate accuracy as the number of correct answers divided by the total number of attempts
accuracy = correct / all if all > 0 else 0  # Ensure division by zero is handled

print(f"Accuracy: {accuracy*100:.2f}%")


สมาชิกสภาเทศบาลเมือง
สมาชิกสภาเทศบาลเมือง
ถ้วยรางวัล
ถ้วยรางวัล
ชูใจ
ชูใจ
ปิติ
ปิติ
พยาน
พยาน
รถส่งของ
รถส่งของ
สมศักดิ์
สมศักดิ์
ชายคนนั้น
ลูกชาย
ลูกบอลอันใหญ่
โต๊ะ
สมจิต
สมเกียรติ
สมบัตร
สมบัตร
ตะวัน
ตะวัน
หุ่นตัวนั้น
หุ่นตัวนั้น
ภาพวาดของขิม
ภาพวาดของขิม
น้ำค้าง
น้ำค้าง
ตำรวจ
นักดับเพลิง
ภัทร
ภัทร
ภีม
ภีม
กระสอบมันฝรั่ง
กระสอบมันฝรั่ง
ภาคิน
ภาคิน
นักเรียนที่โตกว่า
เด็กที่เด็กกว่า
แก้ว
แก้ว
พู่กัน
พู่กัน
ปิติ
ปิติ
สมใจ
สมใจ
ลุงของตะวัน
ลุงของตะวัน
ต้นโอ๊ค
ต้นโอ๊ค
ช่องว่าง
ช่องว่าง
ท่อระบายน้ำ
เส้นผม
รถไฟ
รถไฟ
เวที
เสา
รถไฟใต้ดิน
รถไฟใต้ดิน
ฝน
ฝน
ผ้าขี้ริ้วเก่าๆ
ผ้าขี้ริ้วเก่าๆ
มะลิ
มะลิ
ขวดน้ำ
ขวดน้ำ
ชั้นวาง
ชั้นวาง
อาคาร
อาคาร
มานะ
มานะ
มานะ
มานะ
สมปอง
สมปอง
ปิติ
ปิติ
นักต้มตุ๋น
นักต้มตุ๋น
สุนัข
สุนัข
แมว
แมว
บุตรสาวของแก้ว
บุตรสาวของแก้ว
ลูกสาวของแพร
ลูกสาวของแพร
ชายในชุดทหาร
ชายในชุดทหาร
ปลา
ปลา
หมากฝรั่ง
หมากฝรั่ง
สุนัข
แมว
ต้นไม้
ต้นไม้
พนักงาน
พนักงาน
สมพงษ์
ชายท่านหนึ่ง
ชายผู้เล่นจักกลิ้ง
ชายผู้เล่นจักกลิ้ง
ภูผา
ภูผา
ผู้ปกครองของยุ้ย
ผู้ปกครองของยุ้ย
โดม
โดม
บ้านหลังใหม่
บ้านห

In [ ]:
import anthropic
import time
import requests

client = anthropic.Anthropic(
    api_key="API_KEY"
)

ans = []  # List to store generated responses
correct = 0  # Counter for correctly matched responses
all = 0  # Counter for all attempts
system_prompt = system_prompt_th
model = "claude-3-sonnet-20240229"
for i, p in enumerate(prompt_thai):
    time.sleep(5)
    ret = ask_claude(model,system_prompt,p)[0].text
    print(ret)
    ans.append(ret)  # Store the generated response

    # Determine the gold label based on the correct option in wsc_th
    if label_thai[i] == 0:
        gold_label = option_thai[i][0]
    else:
        gold_label = option_thai[i][1]

    # Check if the generated response matches the gold label
    if ret == gold_label:
        correct += 1  # Increment correct count if there's a match

    all += 1  # Increment total attempts

# Calculate accuracy as the number of correct answers divided by the total number of attempts
accuracy = correct / all if all > 0 else 0  # Ensure division by zero is handled

print(f"Accuracy: {accuracy*100:.2f}%")


สมาชิกสภาเทศบาลเมือง
ผู้ชุมนุม
ถ้วยรางวัล
ถ้วยรางวัล
มานี
มานี
ปิติ
ปิติ
พยาน
พยาน
รถส่งของ
รถโรงเรียน
สมชาย
สมชาย
ชายคนนั้น
ลูกชาย
ลูกบอลอันใหญ่
ลูกบอลอันใหญ่
สมเกียรติ
สมเกียรติ
สมบูรณ์
สมบูรณ์
ตะวัน
ต้องตา
หุ่นตัวนั้น
หุ่นตัวนั้น
ภาพวาดของขิม
ภาพวาดของขิม
น้ำฝน
น้ำค้าง
ตำรวจ
นักดับเพลิง
ภัทร
ภัทร
กรณ์
ภีม
กระสอบมันฝรั่ง
กระสอบมันฝรั่ง
ภาคิน
ออมสิน
นักเรียนที่โตกว่า
เด็กที่เด็กกว่า
แก้ว
แก้ว
จันทร์เจ้า
พู่กัน
ปิติ
ปิติ
สมศรี
สมใจ
ลุงของตะวัน
ลุงของตะวัน
ต้นโอ๊ค
ต้นโอ๊ค
ช่องว่าง
ผนัง
ท่อระบายน้ำ
เส้นผม
รถไฟ
รถไฟ
เสา
เวที
รถไฟใต้ดิน
รถไฟใต้ดิน
ฝน
ฝน
ผ้าขี้ริ้วเก่าๆ
มีด
กระถิน
กระถิน
ขวดน้ำ
ขวดน้ำ
ชั้นวาง
ชั้นวาง
อาคาร
อาคาร
มานะ
มานะ
มานะ
มานะ
สมศักดิ์
สมปอง
ปิติ
ปิติ
นักต้มตุ๋น
สมรัก
สุนัข
ที่ใต้ต้นไม้
แมว
แมว
บุตรสาวของแก้ว
บุตรสาวของแก้ว
ลูกสาวของแพร
ลูกสาวของแพร
ชายในชุดทหาร
ชายในชุดทหาร
ปลา
หนอน
หมากฝรั่ง
หมากฝรั่ง
สุนัข
แมว
ต้นไม้
หลังคา
ลูกค้า
พนักงาน
ชายท่านหนึ่ง
ชายท่านหนึ่ง
ชายผู้เล่นจักกลิ้ง
ชายผู้เล่นจักกลิ้ง
ภานุ
ภานุ
ผู้ปกครองของยุ้ย
ผู้ปกครองของยุ้ย
โดม
โดม
บ้านของตัวเอง

In [ ]:
import anthropic
import time
import requests

client = anthropic.Anthropic(
    api_key="API_KEY"
)

ans = []  # List to store generated responses
correct = 0  # Counter for correctly matched responses
all = 0  # Counter for all attempts
system_prompt = system_prompt_th
model = "claude-3-opus-20240229"
for i, p in enumerate(prompt_thai):
    time.sleep(5)
    ret = ask_claude(model,system_prompt,p)[0].text
    print(ret)
    ans.append(ret)  # Store the generated response

    # Determine the gold label based on the correct option in wsc_th
    if label_thai[i] == 0:
        gold_label = option_thai[i][0]
    else:
        gold_label = option_thai[i][1]

    # Check if the generated response matches the gold label
    if ret == gold_label:
        correct += 1  # Increment correct count if there's a match

    all += 1  # Increment total attempts

# Calculate accuracy as the number of correct answers divided by the total number of attempts
accuracy = correct / all if all > 0 else 0  # Ensure division by zero is handled

print(f"Accuracy: {accuracy*100:.2f}%")


สมาชิกสภาเทศบาลเมือง
สมาชิกสภาเทศบาลเมือง
ถ้วยรางวัล
กระเป๋าเดินทาง
ชูใจ
มานี
ปิติ
ปิติ
ทนายความ
พยาน
รถส่งของ
รถโรงเรียน
สมศักดิ์
สมศักดิ์
ชายคนนั้น
ลูกชาย
ลูกบอลอันใหญ่
ลูกบอลอันใหญ่
สมเกียรติ
สมจิต
สมบูรณ์
สมบูรณ์
ตะวัน
ต้องตา
หุ่นตัวนั้น
ชั้นวาง
ภาพวาดของขิม
ภาพวาดของขิม
น้ำฝน
น้ำค้าง
นักดับเพลิง
ตำรวจ
นนท์
ภัทร
กรณ์
ภีม
กระสอบมันฝรั่ง
กระสอบมันฝรั่ง
ออมสิน
ออมสิน
นักเรียนที่โตกว่า
เด็กที่เด็กกว่า
ขวด
แก้ว
จันทร์เจ้า
พู่กัน
ปิติ
มานะ
สมศรี
สมใจ
ตะวัน
ตะวัน
ต้นโอ๊ค
ต้นโอ๊ค
ช่องว่าง
ช่องว่าง
ท่อระบายน้ำ
เส้นผม
การประชุม
การประชุม
เสา
เวที
เสียงตามสาย
รถไฟใต้ดิน
คอนเสิร์ต
ฝน
ผ้าขี้ริ้วเก่าๆ
มีด
มะลิ
มะลิ
ขวดน้ำ
กระเป๋า
ชั้นวาง
ชั้นวาง
อาคาร
อาคาร
ปิติ
มานะ
ปิติ
มานะ
สมศักดิ์
สมศักดิ์
มานะ
ปิติ
นักต้มตุ๋น
สมรัก
สุนัข
สุนัข
แมว
แมว
แก้ว
บุตรสาวของแก้ว
แพร
แพร
นพ
นพ
ปลา
ปลา
กุญแจ
หมากฝรั่ง
สุนัข
แมว
ต้นไม้
หลังคา
ลูกค้า
พนักงาน
สมพงษ์
ชายท่านหนึ่ง
ธนากร
ชายผู้เล่นจักกลิ้ง
ภานุ
ภานุ
เอ็มและยุ้ย
ผู้ปกครองของยุ้ย
โดม
โดม
บ้านของตัวเอง
บ้านหลังใหม่
คน
งานประพันธ์ของเจนภพ
ขลุ่ย
เพลง
เก้าอี้
เ

## Gemini

In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 7.6 MB/s eta 0:00:00


In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Used to securely store your API key
from google.colab import userdata
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
#GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key="API_KEY")
generation_config={
    "temperature":0
}
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]
model = genai.GenerativeModel('gemini-pro')
ans=[]
all=0
correct=0

for i, p in enumerate(prompt):
    ret = model.generate_content(
    f"{system_prompt}\n{p}",safety_settings=safety_settings)
    ret = to_markdown(ret.text).data.strip('\'>').strip()
    print(ret)
    time.sleep(15)
    ans.append(ret)  # Store the generated response
    # Determine the gold label based on the correct option in wsc_th
    if wsc_th[i]['Label'] == 0:
        gold_label = wsc_th[i]['Option 0']
    else:
        gold_label = wsc_th[i]['Option 1']

    # Check if the generated response matches the gold label
    if ret == gold_label:
        correct += 1  # Increment correct count if there's a match

    all += 1  # Increment total attempts

# Calculate accuracy as the number of correct answers divided by the total number of attempts
accuracy = correct / all if all > 0 else 0  # Ensure division by zero is handled

print(f"Accuracy: {accuracy*100:.2f}%")

ผู้ชุมนุม
ถ้วยรางวัล
ถ้วยรางวัล
มานี
มานี
ปิติ
เขา:ปิติ
พยาน
พยาน
รถส่งของ
รถส่งของ
สมชาย
สมชาย
ลูกชาย
ลูกชาย
ลูกบอลอันใหญ่
ลูกบอลอันใหญ่
สมเกียรติ
สมเกียรติ
สมบัตร
สมบูรณ์
เธอ:ตะวัน
เธอ:ตะวัน
หุ่นตัวนั้น
หุ่นตัวนั้น
ภาพวาดของปลื้ม
ภาพวาดของขิม
น้ำฝน
น้ำฝน
นักดับเพลิง
นักดับเพลิง
เข
ภัทร
กรณ์
กรณ์
กระสอบมันฝรั่ง
กระสอบมันฝรั่ง
ภาคิน
ภาคิน
นักเรียนที่ี่โตกว่า
เด็กที่เด็กกว่า
แก้ว
แก้ว
จันทร์เจ้า
เธอ:พู่กัน
ปิติ
มานะ
สมใจ
เธอ:สมศรี
ตะวัน
ลุงของตะวัน
ภาพวาด
ภาพวาด
ช่องว่าง
ช่องว่าง
เส้นผม
ท่อระบายน้ำ
รถไฟ
รถไฟ
เสา
เสา
เสียงตามสาย
รถไฟใต้ดิน
ฝน
ฝน
ผ้าขี้ริ้วเก่าๆ
มีด
เธอ:มะลิ
เธอ:กระถิน
ขวดน้ำ
ขวดน้ำ
หม้อ
หม้อ
แผนที่
อาคาร
ปิติ
ปิติ
ปิติ
มานะ
สมปอง
สมปอง
มานะ
ปิติ
นักต้มตุ๋น
นักต้มตุ๋น
สุนัข
สุนัข
แมว
แมว
แก้ว:บุตรสาวของแก้ว
เธอ:บุตรสาวของแก้ว
เธอ:แพร
เธอ:แพร
ชายในชุดทหาร
นพ
ปลา
ปลา
หมากฝรั่ง
หมากฝรั่ง
สุนัข
แมว
ต้นไม้
ต้นไม้
พนักงาน
ลูกค้า
ชายท่านหนึ่ง
ชายท่านหนึ่ง
เขา: ธนากร
ชายผู้เล่นจักกลิ้ง
ภานุ
ภานุ
ผู้ปกครองของยุ้ย
ผู้ปกครองของยุ้ย
โดม
โอ๋๋น
บ้านของตัวเอง
บ้านหลังใหม่
คน
คน
ขลุ่ย
ขล

ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 254.36ms


ดอน
ดอน
กลุ่มกรรมการ
กลุ่มกรรมการ


ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 278.48ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1035.24ms


Inwza55YO
มัน:GrWQWu8JyC
Accuracy: 44.01%


In [ ]:
# Used to securely store your API key
from google.colab import userdata
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
#GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key="KEY")
generation_config={
    "temperature":0
}
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]
model = genai.GenerativeModel('gemini-pro')
ans=[]
all=0
correct=0

for i, p in enumerate(prompt_english):
    ret = model.generate_content(
    f"{system_prompt_en}\n{p}",safety_settings=safety_settings)
    ret = to_markdown(ret.text).data.strip('\'>').strip()
    ans.append(ret)  # Store the generated response
    # Determine the gold label based on the correct option in wsc_th
    if label_english[i] == 0:
        gold_label = option_english[i][0]
    else:
        gold_label = option_english[i][1]

    # Check if the generated response matches the gold label
    if ret == gold_label:
        correct += 1  # Increment correct count if there's a match

    all += 1  # Increment total attempts

# Calculate accuracy as the number of correct answers divided by the total number of attempts
accuracy = correct / all if all > 0 else 0  # Ensure division by zero is handled

print(f"Accuracy: {accuracy*100:.2f}%")

ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 809.28ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 884.46ms


KeyboardInterrupt: 

In [ ]:
ans

['The demonstrators',
 'The demonstrators',
 'the trophy',
 'the trophy',
 'Joan',
 'Joan',
 'George',
 'George',
 'the witness',
 'the witness',
 'the delivery truck',
 'the delivery truck',
 'Bill',
 'Frank',
 'The man',
 'The man',
 'The large ball',
 'The large ball',
 'John',
 'John',
 'Tom',
 'Tom',
 'Sue',
 'Sue',
 'The sculpture',
 'The sculpture',
 "Tina's drawing",
 "it:Tina's drawing",
 'Anna',
 'Anna',
 'The firemen',
 'The firemen',
 'Tom',
 'Tom',
 'Jim',
 'Kevin',
 'The sack of potatoes',
 'The sack of potatoes',
 'Pete',
 'Martin',
 'The younger students',
 'The younger students',
 'the cup',
 'the cup',
 'Ann',
 'Ann',
 'Mark',
 'Sid',
 'Susan',
 'Ann',
 'Joe',
 'Joe',
 'The painting',
 'The painting',
 'The gap',
 'The gap',
 'The drain',
 'The drain',
 'The meeting',
 'The train',
 'The pillar',
 'The pillar',
 'The announcement',
 'The subway',
 'The rain',
 'The rain',
 'The rag',
 'The rag',
 'Ann',
 'Ann',
 'The water bottle',
 'The water bottle',
 'The pot',
 'T

## Aya

In [ ]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.8 MB/s eta 0:00:00


In [ ]:
def ask_aya(model,systemPrompt,userPrompt):

    try:
        # Make an API call to OpenAI
        response = co.chat(
            model=model,
            max_tokens=50,
            temperature=0.0,
            seed=77,
            preamble=systemPrompt,
            message=userPrompt,
        )
        return response.text
    except Exception as e:
        return f"An error occurred: {e}"


In [ ]:
import cohere
co = cohere.Client(api_key="API_KEY")

import csv
import os
import anthropic
import time
import requests

# Function to test the model on a specific dataset
def test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name):
    ans_aya = []  # List to store generated responses
    results = []  # List to store results (answer, gold label, and whether it was correct)
    correct = 0  # Counter for correctly matched responses
    total = len(prompts)  # Total number of attempts

    for i, p in enumerate(prompts):
        # Call the model to get a response
        ret = ask_aya(model_name, system_prompt, p)
        ans_aya.append(ret)  # Store the generated response

        # Determine the gold label based on the correct option
        if labels[i] == 0:
            gold_label = options[i][0]
        else:
            gold_label = options[i][1]

        # Check if the generated response matches the gold label
        is_correct = int(ret == gold_label)
        correct += is_correct  # Increment correct count if there's a match
        results.append((dataset_name, ret, gold_label, is_correct))  # Store result (dataset, answer, gold label, correct/incorrect)

    # Calculate accuracy
    accuracy = correct / total if total > 0 else 0

    # Print and return the results
    print(f"Dataset: {dataset_name}, Accuracy: {accuracy * 100:.2f}%")
    return results, accuracy

# Dataset configurations
datasets = [
    {"name": "dataset_en", "prompts": prompt_english, "options": option_english, "labels": label_english},
    {"name": "dataset_th", "prompts": prompt_thai, "options": option_thai, "labels": label_thai},
    {"name": "dataset_th_exact", "prompts": prompt_thai_exact, "options": option_thai_exact, "labels": label_thai_exact},
]

model_name = "command-r-plus-08-2024"
system_prompt = system_prompt_en

# Initialize a CSV file for each model that will hold the results from all datasets
with open(f'{model_name}_results.csv', mode='w', newline='', encoding='utf-8') as model_results_file:
    csv_writer = csv.writer(model_results_file)

    # Write the header of the CSV file
    csv_writer.writerow(['Dataset', 'Model Answer', 'Gold Label', 'Correct'])

    # Loop through each dataset and test the model
    for dataset in datasets:
        dataset_name = dataset["name"]
        prompts = dataset["prompts"]
        options = dataset["options"]
        labels = dataset["labels"]

        # Test the model on the current dataset
        results, accuracy = test_model_on_dataset(model_name, system_prompt, prompts, options, labels, dataset_name)

        # Write the dataset results to the CSV file
        for result in results:
            csv_writer.writerow(result)

print(f"Results have been saved to {model_name}_results.csv")



Dataset: dataset_en, Accuracy: 87.02%
Dataset: dataset_th, Accuracy: 61.75%
Dataset: dataset_th_exact, Accuracy: 64.91%
Results have been saved to command-r-plus-08-2024_results.csv
